Imports Libraries

In [40]:
import pandas as pd
from IPython.display import display
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



Loads Data Frame

In [41]:
with open("BTCUSD_15min_tagged.pickle", \
          "rb") as f:
      df \
      = pickle.load(f)

df = df.drop(['Low','Close','High'], axis=1)
df
#df.drop(df.tail(1).index,inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'BTCUSD_15min_tagged.pickle'

In [33]:
df_train = df.sample(frac=0.7, random_state=0)
df_valid = df.drop(df_train.index)

max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)



df_train = (df_train - min_) / (max_ - min_)
df_valid = (df_valid - min_) / (max_ - min_)

X_train = df_train.drop('Position', axis=1)
X_valid = df_valid.drop('Position', axis=1)
y_train = df_train['Position']
y_valid = df_valid['Position']



Loads a model

In [34]:
model = tf.keras.models.load_model('Classifier_NN_hourly_v2.0')

# Check its architecture

model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 8)                 96        
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 177
Trainable params: 177
Non-trainable params: 0
_________________________________________________________________


Creates a Model

In [18]:

model = keras.Sequential([
    layers.Dense(8, activation='relu', input_shape=[11]),
    layers.Dense(8, activation='relu'),    
    layers.Dense(1, activation='sigmoid'),
])

In [79]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

Trains a model

In [83]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)



history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=64,
    epochs=5000,
    #callbacks=[early_stopping],
    verbose=0, # hide the output because we have so many epochs
)


In [36]:
score = model.evaluate(X_valid, y_valid, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 82.81367492675781
Test accuracy: 0.5183033347129822


In [21]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[5:, ['loss', 'val_loss']].plot()
history_df.loc[5:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_binary_accuracy'].max()))

NameError: name 'history' is not defined

In [25]:
model.save('Classifier_30min_v2.0')

AttributeError: 'VotingClassifier' object has no attribute 'save'

In [38]:
import sklearn.naive_bayes
import sklearn.feature_extraction
import sklearn.preprocessing
import sklearn.linear_model
import sklearn.metrics
import time
import sklearn.neighbors
import sklearn.ensemble
import sklearn.metrics
#pip install xgboost
import xgboost

In [39]:
# Creates a model

# Create naive bayes classifier
naive_bayes = sklearn.naive_bayes.MultinomialNB()

# Create a logistic regression classifier
logistic = sklearn.linear_model.LogisticRegression(solver='newton-cg',
                                                   tol=1e-2, max_iter=10000)

#Create a xgboost classifier
#xgboost = xgboost.XGBRFClassifier(alpha=1, max_depth=30)

# Create a k-Nearest neighbors classifier
kNN = sklearn.neighbors.KNeighborsClassifier(n_neighbors=150)

# Create a Random Forest classifier
randforest = sklearn.ensemble.RandomForestClassifier(n_estimators=1000,
    min_samples_leaf=0.003)


# Create a voting ensemble of classifiers
model = sklearn.ensemble.VotingClassifier(
    estimators=[('logistic', logistic),
                #('xgboost', xgboost),
                #('kNN', kNN),
                ('naive_bayes',naive_bayes),
                ('randforest', randforest),
                ],voting='soft')


# Train it with the training data and labels
model.fit(X_train, y_train)

# Predict
pred = model.predict(X_valid)


# Metrics
#

# Confusion matrix
cmatrix = sklearn.metrics.confusion_matrix(y_valid, pred)
print("Confusion Matrix:")
print(cmatrix)

# Accuracy, precision & recall
print("Accuracy:   {:.3f}".format(sklearn.metrics.accuracy_score(y_valid, pred)))
print("Precision:  {:.3f}".format(sklearn.metrics.precision_score(y_valid, pred, average='weighted')))
print("Recall:     {:.3f}".format(sklearn.metrics.recall_score(y_valid, pred, average='weighted')))



Confusion Matrix:
[[403 436]
 [239 643]]
Accuracy:   0.608
Precision:  0.611
Recall:     0.608


In [ ]:
model.